In [13]:
# Install required libraries
!pip install streamlit yfinance pandas numpy matplotlib seaborn scikit-learn pyngrok --quiet

import streamlit as st
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os
from pyngrok import ngrok

# ✅ Set up ngrok authentication
NGROK_AUTH_TOKEN = "2t63vH7Kod7pqQFgkQVyoD1t5QF_72BpKo37zzsdg3pUNz5C1"  # Use your actual token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Function to fetch stock data
def fetch_stock_data(ticker, start_date, end_date):
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty or "Close" not in data.columns:
            return None  # Return None if no valid data
        return data
    except Exception as e:
        return None

# Function to compute investment metrics
def compute_metrics(data):
    data['Daily Return'] = data['Close'].pct_change()
    annual_return = data['Daily Return'].mean() * 252
    annual_volatility = data['Daily Return'].std() * np.sqrt(252)
    sharpe_ratio = annual_return / annual_volatility if annual_volatility != 0 else 0
    return annual_return, annual_volatility, sharpe_ratio

# Function to predict future returns
def predict_future_returns(data):
    data = data.dropna()
    data['Days'] = np.arange(len(data))
    X = data[['Days']]
    y = data['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return model, mse

# Streamlit Web App
def main():
    st.title("📈 Investment Portfolio Analysis & Prediction")

    # User Inputs
    ticker = st.text_input("Enter Stock Ticker (e.g., AAPL, TSLA, AMZN):", "AAPL")
    start_date = st.date_input("Start Date", pd.to_datetime("2022-01-01"))
    end_date = st.date_input("End Date", pd.to_datetime("2024-01-01"))

    if st.button("Analyze Investment"):
        with st.spinner("Fetching data..."):
            data = fetch_stock_data(ticker, start_date, end_date)
            if data is None:
                st.error(f"❌ No data found for ticker '{ticker}'. Check the symbol and date range.")
                return

            if 'Close' not in data.columns:
                st.error("❌ 'Close' column is missing. Unable to analyze this stock.")
                return

            st.success("✅ Data Fetched Successfully!")

            # Display Data
            st.subheader("📊 Stock Price Data (Last 5 Days)")
            st.dataframe(data.tail())

            # Plot Closing Price
            st.subheader("📈 Stock Closing Price Over Time")
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.plot(data.index, data['Close'], label="Closing Price")
            ax.set_title(f"{ticker} Closing Price")
            ax.set_xlabel("Date")
            ax.set_ylabel("Price (USD)")
            ax.legend()
            st.pyplot(fig)

            # Compute Metrics
            annual_return, annual_volatility, sharpe_ratio = compute_metrics(data)
            st.subheader("📊 Investment Metrics")
            st.write(f"**Annual Return:** {annual_return:.2%}")
            st.write(f"**Annual Volatility:** {annual_volatility:.2%}")
            st.write(f"**Sharpe Ratio:** {sharpe_ratio:.2f}")

            # Prediction
            st.subheader("🔮 Predict Future Returns")
            model, mse = predict_future_returns(data)
            future_days = np.array(range(len(data), len(data) + 30)).reshape(-1, 1)
            future_prices = model.predict(future_days)

            fig2, ax2 = plt.subplots(figsize=(12, 6))
            ax2.plot(future_days, future_prices, label="Predicted Price", color="red")
            ax2.set_title(f"{ticker} Predicted Future Prices")
            ax2.set_xlabel("Days")
            ax2.set_ylabel("Price (USD)")
            ax2.legend()
            st.pyplot(fig2)

            st.write(f"**Model Mean Squared Error:** {mse:.2f}")

if __name__ == "__main__":
    # Save Streamlit app to a file
    with open("app.py", "w") as f:
        f.write("""
import streamlit as st
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data if 'Close' in data.columns else None

def compute_metrics(data):
    data['Daily Return'] = data['Close'].pct_change()
    annual_return = data['Daily Return'].mean() * 252
    annual_volatility = data['Daily Return'].std() * np.sqrt(252)
    sharpe_ratio = annual_return / annual_volatility if annual_volatility != 0 else 0
    return annual_return, annual_volatility, sharpe_ratio

def predict_future_returns(data):
    data = data.dropna()
    data['Days'] = np.arange(len(data))
    X = data[['Days']]
    y = data['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model, mean_squared_error(y_test, model.predict(X_test))

def main():
    st.title("📈 Investment Portfolio Analysis & Prediction")
    ticker = st.text_input("Enter Stock Ticker:", "AAPL")
    start_date = st.date_input("Start Date", pd.to_datetime("2022-01-01"))
    end_date = st.date_input("End Date", pd.to_datetime("2024-01-01"))

    if st.button("Analyze Investment"):
        data = fetch_stock_data(ticker, start_date, end_date)
        if data is not None:
            st.line_chart(data['Close'])
            annual_return, annual_volatility, sharpe_ratio = compute_metrics(data)
            st.write(f"Annual Return: {annual_return:.2%}")
            st.write(f"Annual Volatility: {annual_volatility:.2%}")
            st.write(f"Sharpe Ratio: {sharpe_ratio:.2f}")
            model, mse = predict_future_returns(data)
            st.write(f"Model Mean Squared Error: {mse:.2f}")
        else:
            st.error("Invalid stock ticker or date range.")

if __name__ == "__main__":
    main()
""")

    # ✅ Run the Streamlit app and expose it with ngrok
    os.system("streamlit run app.py &")
    public_url = ngrok.connect(8501).public_url
    print(f"🔗 Streamlit App is live: {public_url}")


🔗 Streamlit App is live: https://64e4-34-21-53-136.ngrok-free.app
